In [1]:
import logging
import asyncio
import pandas as pd
from IPython.display import display, Markdown

from datetime import datetime

import influxdb_client

from tastytrade.common.logging import setup_logging
from tastytrade.config.enumerations import Channels
from tastytrade.connections.sockets import DXLinkManager
from tastytrade.connections import Credentials, InfluxCredentials

from tastytrade.messaging.processors import TelegrafHTTPEventProcessor, RedisEventProcessor
from tastytrade.config import RedisConfigManager
from tastytrade.connections.subscription import RedisSubscriptionStore
from tastytrade.utils.time_series import forward_fill

# Show all rows in pandas DataFrames
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

logging.getLogger().handlers.clear()

TEST = True
ENV = "Live"
DURATION = 15

EDT = 5

start_time = datetime(2025, 1, 1)

setup_logging(
    level=logging.INFO,
    log_dir="../logs",
    filename_prefix=f"{'dev' if TEST else 'prod'}_tastytrade",
    console=True,
    file=True,
)

loop = asyncio.get_event_loop()
loop.set_debug(True)
logging.getLogger("asyncio").setLevel(logging.DEBUG)

2025-08-05 22:19:05 - INFO:root:62:Logging initialized - writing to ../logs/dev_tastytrade_20250805.log


# Service Connections

In [ ]:
config = RedisConfigManager(env_file="/workspace/.env")
config.initialize(force=True)

credentials = Credentials(config=config, env="Live")

dxlink = DXLinkManager(subscription_store=RedisSubscriptionStore())
await dxlink.open(credentials=credentials)

influx_user = InfluxCredentials(config=config)
influxdb = influxdb_client.InfluxDBClient(
    url=influx_user.url, token=influx_user.token, org=influx_user.org
)

for handler in dxlink.router.handler.values():
    handler.add_processor(TelegrafHTTPEventProcessor())
    handler.add_processor(RedisEventProcessor())

2025-08-05 22:19:09 - INFO:tastytrade.config.manager:174:Initialized 19 variables from .env file in Redis
2025-08-05 22:19:09 - DEBUG:asyncio:848:Get address info api.tastyworks.com:443, type=<SocketKind.SOCK_STREAM: 1>, flags=<AddressInfo.AI_ADDRCONFIG: 32>
2025-08-05 22:19:09 - INFO:asyncio:856:Getting address info api.tastyworks.com:443, type=<SocketKind.SOCK_STREAM: 1>, flags=<AddressInfo.AI_ADDRCONFIG: 32> took 357.721ms: [(<AddressFamily.AF_INET: 2>, <SocketKind.SOCK_STREAM: 1>, 6, '', ('170.76.244.141', 443))]
2025-08-05 22:19:09 - DEBUG:asyncio:535:<asyncio.sslproto.SSLProtocol object at 0x7c8efc1fcd10> starts SSL handshake
2025-08-05 22:19:09 - DEBUG:asyncio:594:<asyncio.sslproto.SSLProtocol object at 0x7c8efc1fcd10>: SSL handshake took 44.2 ms
2025-08-05 22:19:09 - DEBUG:asyncio:1121:<asyncio.TransportSocket fd=83, family=2, type=1, proto=6, laddr=('172.18.0.7', 51564), raddr=('170.76.244.141', 443)> connected to None:None: (<asyncio.sslproto._SSLProtocolTransport object at 0

2025-08-05 22:19:10 - INFO:tastytrade.messaging.handlers:225:SETUP
2025-08-05 22:19:10 - INFO:tastytrade.messaging.handlers:228:AUTH_STATE:UNAUTHORIZED
2025-08-05 22:19:10 - INFO:tastytrade.messaging.handlers:228:AUTH_STATE:AUTHORIZED
2025-08-05 22:19:10 - INFO:tastytrade.messaging.handlers:231:CHANNEL_OPENED:1
2025-08-05 22:19:10 - INFO:tastytrade.messaging.handlers:231:CHANNEL_OPENED:3
2025-08-05 22:19:10 - INFO:tastytrade.messaging.handlers:231:CHANNEL_OPENED:5
2025-08-05 22:19:10 - INFO:tastytrade.messaging.handlers:231:CHANNEL_OPENED:7
2025-08-05 22:19:10 - INFO:tastytrade.messaging.handlers:231:CHANNEL_OPENED:9
2025-08-05 22:19:10 - INFO:tastytrade.messaging.handlers:231:CHANNEL_OPENED:11
2025-08-05 22:19:10 - INFO:tastytrade.messaging.handlers:231:CHANNEL_OPENED:99
2025-08-05 22:19:10 - INFO:tastytrade.messaging.handlers:236:FEED_CONFIG:5:COMPACT
2025-08-05 22:19:10 - INFO:tastytrade.messaging.handlers:236:FEED_CONFIG:7:COMPACT
2025-08-05 22:19:10 - INFO:tastytrade.messaging.han

# Market Data Subscriptions

In [3]:
start_time = datetime(2025, 8, 5)

symbols = ["BTC/USD:CXTALP", "NVDA", "AAPL", "QQQ", "SPY", "SPX"]  # , "/ESH25:XCME"]
# symbols = ["BTC/USD:CXTALP", "NVDA",]
# symbols = ["AAPL", "QQQ"]  # 
# symbols = ["SPY", "SPX"]  # 
intervals = ["1d", "1h", "30m", "15m", "5m", "m"]

# ticker subscriptions
await dxlink.subscribe(symbols)

# candle subscriptions
for symbol in symbols:
    for interval in intervals:
        coroutine = dxlink.subscribe_to_candles(
            symbol=symbol,
            interval=interval,
            from_time=start_time,
        )
        await asyncio.wait_for(coroutine, timeout=10)

2025-08-05 22:19:20 - INFO:tastytrade.connections.sockets:236:Added subscription: BTC/USD:CXTALP
2025-08-05 22:19:20 - INFO:tastytrade.connections.sockets:236:Added subscription: NVDA
2025-08-05 22:19:20 - INFO:tastytrade.connections.sockets:236:Added subscription: AAPL
2025-08-05 22:19:20 - INFO:tastytrade.connections.sockets:236:Added subscription: QQQ
2025-08-05 22:19:20 - INFO:tastytrade.connections.sockets:236:Added subscription: SPY
2025-08-05 22:19:20 - INFO:tastytrade.connections.sockets:236:Added subscription: SPX
2025-08-05 22:19:20 - INFO:tastytrade.connections.sockets:236:Added subscription: BTC/USD:CXTALP{=d}
2025-08-05 22:19:20 - INFO:tastytrade.connections.sockets:236:Added subscription: BTC/USD:CXTALP{=h}
2025-08-05 22:19:20 - INFO:tastytrade.connections.sockets:236:Added subscription: BTC/USD:CXTALP{=30m}
2025-08-05 22:19:20 - INFO:tastytrade.connections.sockets:236:Added subscription: BTC/USD:CXTALP{=15m}
2025-08-05 22:19:20 - INFO:tastytrade.connections.sockets:236:A

In [4]:
# forward fill
for symbol in symbols:
    for interval in intervals:
        event_symbol = f"{symbol}{{={interval}}}"
        logging.debug("Forward-filling %s", event_symbol)
        forward_fill(symbol=event_symbol, lookback_days=5)

2025-08-05 22:19:25 - INFO:root:120:Successfully flushed and closed InfluxDB write API
2025-08-05 22:19:25 - INFO:root:124:Forward-fill added 1 events for BTC/USD:CXTALP{=30m}
2025-08-05 22:19:25 - INFO:root:120:Successfully flushed and closed InfluxDB write API
2025-08-05 22:19:25 - INFO:root:124:Forward-fill added 2 events for BTC/USD:CXTALP{=15m}
2025-08-05 22:19:25 - INFO:root:120:Successfully flushed and closed InfluxDB write API
2025-08-05 22:19:25 - INFO:root:124:Forward-fill added 17 events for BTC/USD:CXTALP{=5m}
2025-08-05 22:19:26 - INFO:root:120:Successfully flushed and closed InfluxDB write API
2025-08-05 22:19:26 - INFO:root:124:Forward-fill added 209 events for BTC/USD:CXTALP{=m}
2025-08-05 22:19:26 - INFO:root:120:Successfully flushed and closed InfluxDB write API
2025-08-05 22:19:26 - INFO:root:124:Forward-fill added 2 events for NVDA{=d}
2025-08-05 22:19:26 - INFO:root:120:Successfully flushed and closed InfluxDB write API
2025-08-05 22:19:26 - INFO:root:124:Forward-f

# Check | Market Data feeds

In [5]:
symbol = "SPX{=m}"
symbol = "BTC/USD:CXTALP{=5m}"
# symbol = "/ESH25:XCME{=m}"

display(Markdown(f"**Candle Feed:** {symbol}"))
display(
    dxlink.router.handler[Channels.Candle]
    .processors["feed"]
    .frames[f"{symbol}"]
    .tail(5)
    .sort(by="time", descending=True)
)

**Candle Feed:** BTC/USD:CXTALP{=5m}

shape: (5, 16)
┌────────────┬────────────┬────────────┬────────────┬───┬───────────┬───────────┬──────┬───────────┐
│ eventSymbo ┆ time       ┆ eventFlags ┆ index      ┆ … ┆ askVolume ┆ openInter ┆ vwap ┆ impVolati │
│ l          ┆ ---        ┆ ---        ┆ ---        ┆   ┆ ---       ┆ est       ┆ ---  ┆ lity      │
│ ---        ┆ datetime[μ ┆ i64        ┆ i64        ┆   ┆ f64       ┆ ---       ┆ f64  ┆ ---       │
│ str        ┆ s]         ┆            ┆            ┆   ┆           ┆ f64       ┆      ┆ f64       │
╞════════════╪════════════╪════════════╪════════════╪═══╪═══════════╪═══════════╪══════╪═══════════╡
│ BTC/USD:CX ┆ 2025-08-05 ┆ 0          ┆ 7535228492 ┆ … ┆ null      ┆ 0.0       ┆ null ┆ null      │
│ TALP{=5m}  ┆ 22:15:00   ┆            ┆ 552601600  ┆   ┆           ┆           ┆      ┆           │
│ BTC/USD:CX ┆ 2025-08-05 ┆ 0          ┆ 7535227204 ┆ … ┆ null      ┆ 0.0       ┆ null ┆ null      │
│ TALP{=5m}  ┆ 22:10:00   ┆            ┆ 062412800  ┆   ┆           ┆           ┆      ┆           │
│ BTC/USD:CX ┆ 2025-08-05 ┆ 0          ┆ 7535225915 ┆ … ┆ null      ┆ 0.0       ┆ null ┆ null      │
│ TALP{=5m}  ┆ 22:05:00   ┆            ┆ 572224000  ┆   ┆           ┆           ┆      ┆           │
│ BTC/USD:CX ┆ 2025-08-05 ┆ 0          ┆ 7535224627 ┆ … ┆ null      ┆ 0.0       ┆ null ┆ null      │
│ TALP{=5m}  ┆ 22:00:00   ┆            ┆ 082035200  ┆   ┆           ┆           ┆      ┆           │
│ BTC/USD:CX ┆ 2025-08-05 ┆ 0          ┆ 7535223338 ┆ … ┆ null      ┆ 0.0       ┆ null ┆ null      │
│ TALP{=5m}  ┆ 21:55:00   ┆            ┆ 591846400  ┆   ┆           ┆           ┆      ┆           │
└────────────┴────────────┴────────────┴────────────┴───┴───────────┴───────────┴──────┴───────────┘

In [6]:
display(Markdown(f"**Quote Feed:** {symbol}"))

display(dxlink.router.handler[Channels.Quote].processors["feed"].pl)

**Quote Feed:** BTC/USD:CXTALP{=5m}

shape: (6, 5)
┌────────────────┬───────────┬───────────┬─────────┬─────────┐
│ eventSymbol    ┆ bidPrice  ┆ askPrice  ┆ bidSize ┆ askSize │
│ ---            ┆ ---       ┆ ---       ┆ ---     ┆ ---     │
│ str            ┆ f64       ┆ f64       ┆ f64     ┆ f64     │
╞════════════════╪═══════════╪═══════════╪═════════╪═════════╡
│ NVDA           ┆ 176.92    ┆ 177.0     ┆ 360.0   ┆ 95.0    │
│ BTC/USD:CXTALP ┆ 113752.78 ┆ 114330.03 ┆ 0.01    ┆ 0.05    │
│ AAPL           ┆ 202.64    ┆ 202.71    ┆ 1.0     ┆ 18.0    │
│ SPY            ┆ 626.98    ┆ 627.05    ┆ 100.0   ┆ 500.0   │
│ SPX            ┆ 6234.59   ┆ 6351.3    ┆ null    ┆ null    │
│ QQQ            ┆ 558.23    ┆ 558.29    ┆ 600.0   ┆ 600.0   │
└────────────────┴───────────┴───────────┴─────────┴─────────┘

In [ ]:
# TODOS

# [x] Widen the plot
# [x] Remove the scroller at the bottom
# [x] move the legent and remove Price (that is obvious)
# [x] Add MACD
# [x] Fix HULL - Align w/ candlesticks
# [x] ERROR if no study data found

# [ ] Add RSI
# [ ] Add Volume Profile (?? ... /ES, SPY, etc)
# [ ] Add velocity metric

In [ ]:
await dxlink.close()